In [1]:
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
import seaborn as sns
import numpy as np
import pandas as pd

MODEL_NAME = 'models/yolo12m.pt'
DATASET_YAML = 'data/data.yaml'

TRAIN_PARAMS = {
    'data': DATASET_YAML,
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'name': 'classroom_attention_yolov12m',
    'project': 'yolov12m/train',
    'workers': 4,
    'optimizer': 'SGD',
    'patience': 20,
    'device': '0',
    'save': True,
    'exist_ok': True,
    'verbose': True
}

def train_model(model_path, params):
    print(f"\n Loading Model: {model_path}")
    model = YOLO(model_path)
    print("Resolved model name:", model.model_name)
    
    print("\n Training Started...\n")
    results = model.train(**params)
    
    print("\n Training Complete!")
    return model, results

def evaluate_model(model, dataset_yaml):
    print("\n Running validation on VALIDATION split...\n")
    val_metrics = model.val(data=dataset_yaml, split='val')

    print("\n Validation Metrics:")
    print(f"mAP50: {val_metrics.box.map50:.4f}")
    print(f"mAP50-95: {val_metrics.box.map:.4f}")
    print(f"Precision: {val_metrics.box.mp:.4f}")
    print(f"Recall: {val_metrics.box.mr:.4f}")
    plot_per_class_metrics(
    val_metrics.box.p,
    val_metrics.box.r,
    class_names=['Looking_Forward', 'Raising_Hand', 'Reading', 'Sleeping', 'Turning_Around'],
    save_path='yolov12m/evaluation_plots/validation/per_val_class_metrics.png'
)
    print("\n Running validation on TEST split...\n")
    test_metrics = model.val(data=dataset_yaml, split='test')

    print("\n Test Metrics:")
    print(f"mAP50: {test_metrics.box.map50:.4f}")
    print(f"mAP50-95: {test_metrics.box.map:.4f}")
    print(f"Precision: {test_metrics.box.mp}")
    print(f"Recall: {test_metrics.box.mr}")
    plot_per_class_metrics(
    test_metrics.box.p,
    test_metrics.box.r,
    class_names=['Looking_Forward', 'Raising_Hand', 'Reading', 'Sleeping', 'Turning_Around'],
    save_path='yolov12m/evaluation_plots/validation/per_test_class_metrics.png'
)
    return val_metrics, test_metrics


def plot_metrics(metrics, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    try:
        map50 = metrics.box.map50
        map95 = metrics.box.map
        precision = metrics.box.mp
        recall = metrics.box.mr

        plt.figure(figsize=(10, 6))
        plt.bar(['mAP@50', 'mAP@50-95', 'Precision', 'Recall'],
                [map50, map95, precision, recall],
                color=['dodgerblue', 'teal', 'orange', 'tomato'])

        plt.ylabel('Score')
        plt.ylim(0, 1)
        plt.title('Overall Detection Metrics')
        plt.tight_layout()
        plot_path = os.path.join(save_dir, 'overall_metrics.png')
        plt.savefig(plot_path)
        plt.close()

        print(f" Metric plot saved to {plot_path}")
    except Exception as e:
        print(f" Error while plotting metrics: {e}")


def plot_training_loss_curves(run_dir='yolov12m/detect/exp', save_dir='yolov12m/training_plots'):
    os.makedirs(save_dir, exist_ok=True)

    results_path = os.path.join(run_dir, 'results.csv')
    if not os.path.isfile(results_path):
        print(f" No results.csv found at {results_path}")
        return

    df = pd.read_csv(results_path)

    # Plot available loss curves — check column names if needed
    loss_keys = ['train/box_loss', 'train/cls_loss', 'train/dfl_loss']
    plt.figure(figsize=(10, 6))
    plotted = False
    for key in loss_keys:
        if key in df.columns:
            plt.plot(df.index, df[key], label=key)
            plotted = True

    if plotted:
        plt.title("Training Loss Curves")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid(True)
        plot_path = os.path.join(save_dir, 'loss_curves.png')
        plt.savefig(plot_path)
        plt.close()
        print(f" Training curves saved to {plot_path}")
    else:
        print(" No loss columns found in results.csv")


def plot_predictions(model, save_dir='yolov12m/prediction_outputs'):
    os.makedirs(save_dir, exist_ok=True)

    results = model.predict(
        source="data/test/images",
        save=True,
        save_txt=True,
        conf=0.25,
        max_det=100,
        project='yolov12m/prediction_outputs',
        name='yolov12m-pred'
    )
    print(f"\n Sample predictions saved in {results[0].save_dir}")

def plot_per_class_metrics(precision, recall, class_names=None, save_path=None):
    num_classes = len(precision)
    indices = np.arange(num_classes)

    if class_names is None:
        class_names = [f"Class {i}" for i in range(num_classes)]

    width = 0.35  # Width of bars

    plt.figure(figsize=(12, 6))
    plt.bar(indices - width/2, precision, width, label='Precision', color='skyblue')
    plt.bar(indices + width/2, recall, width, label='Recall', color='salmon')

    plt.xlabel('Class')
    plt.ylabel('Score')
    plt.title('Per-Class Precision and Recall')
    plt.xticks(indices, class_names, rotation=45, ha='right')
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved plot to {save_path}")
    else:
        plt.show()

model, train_results = train_model(MODEL_NAME, TRAIN_PARAMS)

plot_training_loss_curves(run_dir='yolov12m/train/classroom_attention_yolov12m')

val_metrics, test_metrics = evaluate_model(model, DATASET_YAML)
plot_metrics(val_metrics, save_dir='yolov12m/evaluation_plots/validation')
plot_metrics(test_metrics, save_dir='yolov12m/evaluation_plots/test')
plot_predictions(model)


 Loading Model: models/yolo12m.pt
Resolved model name: models/yolo12m.pt

 Training Started...

Ultralytics 8.3.168  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models/yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=classroom_attention_yolov12m, nbs=64, nms=F

train: Scanning D:\research2\classroom-attention\data\train\labels.cache... 1583 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1583/1583 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 499.0124.3 MB/s, size: 42.5 KB)


val: Scanning D:\research2\classroom-attention\data\valid\labels.cache... 453 images, 0 backgrounds, 0 corrupt: 100%|██████████| 453/453 [00:00<?, ?it/s]


Plotting labels to yolov12m\train\classroom_attention_yolov12m\labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 123 weight(decay=0.0), 130 weight(decay=0.0005), 129 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to yolov12m\train\classroom_attention_yolov12m
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        453        453      0.997          1      0.995      0.946
       Looking_Forward        101        101          1          1      0.995       0.96
          Raising_Hand         96         96      0.995          1      0.995      0.945
               Reading         73         73          1          1      0.995      0.953
              Sleeping         81         81      0.995          1      0.995      0.936
        Turning_Around        102        102      0.996          1      0.995      0.936
Speed: 0.1ms preprocess, 4.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to yolov12m\train\classroom_attention_yolov12m

 Training Complete!
 Training curves saved to yolov12m/training_plots\loss_curves.png

 Running validation on VALIDATION split...

Ultralytics 8.3.168  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLOv12m summary (fused): 169 layers, 20,108,767 parameters, 0 gradients, 67.1 GFLOPs

val: Scanning D:\research2\classroom-attention\data\valid\labels.cache... 453 images, 0 backgrounds, 0 corrupt: 100%|██████████| 453/453 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:04<00:00,  6.30it/s]


                   all        453        453      0.995          1      0.995      0.946
       Looking_Forward        101        101          1          1      0.995       0.96
          Raising_Hand         96         96      0.995          1      0.995      0.946
               Reading         73         73          1          1      0.995      0.957
              Sleeping         81         81      0.995          1      0.995      0.936
        Turning_Around        102        102      0.986          1      0.995      0.933
Speed: 0.2ms preprocess, 7.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to yolov12m\train\classroom_attention_yolov12m

 Validation Metrics:
mAP50: 0.9950
mAP50-95: 0.9462
Precision: 0.9953
Recall: 1.0000
Saved plot to yolov12m/evaluation_plots/validation/per_val_class_metrics.png

 Running validation on TEST split...

Ultralytics 8.3.168  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
val: Fast image

val: Scanning D:\research2\classroom-attention\data\test\labels.cache... 226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 226/226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.19it/s]


                   all        226        226      0.999          1      0.995      0.942
       Looking_Forward         36         36          1          1      0.995      0.964
          Raising_Hand         64         64          1          1      0.995       0.95
               Reading         39         39          1          1      0.995      0.954
              Sleeping         33         33      0.997          1      0.995      0.934
        Turning_Around         54         54          1          1      0.995      0.907
Speed: 0.3ms preprocess, 7.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to yolov12m\train\classroom_attention_yolov12m

 Test Metrics:
mAP50: 0.9950
mAP50-95: 0.9419
Precision: 0.999401959341377
Recall: 1.0
Saved plot to yolov12m/evaluation_plots/validation/per_test_class_metrics.png
 Metric plot saved to yolov12m/evaluation_plots/validation\overall_metrics.png
 Metric plot saved to yolov12m/evaluation_plots/test\overall_metrics.png

imag